# Compare 2017 Child Tax Credit to repeal

Investigates why tax units would be worse off when using the 2017 CTC than a repealed CTC, as Sean Wang discovered when running my CTC -> child benefit notebook using the PUF ([notebook](https://github.com/GoFroggyRun/Notebook/blob/master/ctc_ubi_puf.ipynb)).

**Culprit: Non-child dependent credit** which had a different phase-in after the CTC and child dependent credit were set to zero.

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np

In [2]:
tc.__version__

'0.20.1'

### Settings

In [3]:
pd.set_option('precision', 2)

### Utilities

In [4]:
def weighted_sum(df, col):
    return (df[col] * 1.0 * df.s006).sum()

def weighted_mean(df, col):
    return weighted_sum(df, col) / df.s006.sum()

### Create reforms

CTC repeal involves eliminating the normal CTC as well as the new dependent credit for children.

A reform to return to 2017 CTC law is used to show how the distribution of CTC benefits changed with TCJA.

In [5]:
noctc_reform = {
    2018: {
        "_CTC_c": [0],
        "_CTC_ps": [[75000.0, 110000.0, 55000.0, 75000.0, 75000.0]],
#         "_DependentCredit_Nonchild_c": [0.0],
        "_DependentCredit_Child_c": [0.0],
        "_DependentCredit_before_CTC": [False]
    }
}

y2017_reform = {
    2018: {
        # Current: 1400.0
        "_CTC_c": [1000.0],
        # Current: [200000.0, 400000.0, 200000.0, 200000.0, 400000.0]
        "_CTC_ps": [[75000.0, 110000.0, 55000.0, 75000.0, 75000.0]],
        # Current: 2500.0
        "_ACTC_Income_thd": [3000.0],
        # Current: 600.0
        "_DependentCredit_Child_c": [0.0],
        # Current: 500.0
#         "_DependentCredit_Nonchild_c": [0.0],
        # Current: True
        "_DependentCredit_before_CTC": [False]
    }
}

## Generate data

In [6]:
recs = tc.Records.cps_constructor()

In [7]:
def static_baseline_calc(year):
    calc = tc.Calculator(records=recs, policy=tc.Policy())
    calc.advance_to_year(year)
    calc.calc_all()
    return calc

In [8]:
def static_calc(recs,
                ctc_treatment='keep',
                year=2018,
                cols=['s006', 'aftertax_income', 'expanded_income',
                      'n24', 'nu18', 'e18400', 'XTOT', 
                      'c07220',  # CTC
                      'c11070',  # Refunded CTC
                      'dep_credit'  # Dependent credit on top of CTC
                     ]):
    """Creates static Calculator.

    Args:
        ctc_treatment: How the Child Tax Credit is treated. Options include:
            * 'keep': No change. Default.
            * 'repeal': End entirely.
            * 'y2017': Use 2017 law.
        year: Year to advance calculations to.
        cols: Columns to extract per Calculator record. 
            Defaults to ['s006', 'expanded_income', 'aftertax_income', 'nu18',
            'n24', 'XTOT'].
        
    Returns:
        DataFrame with `cols` and percentile, decile, and quintile of 
        after-tax income.
    """
    pol = tc.Policy()
    # Enact reform based on ctc_treatment.
    # Repeal CTC unless it's kept.
    if ctc_treatment == 'y2017':
        pol.implement_reform(y2017_reform)
    elif ctc_treatment == 'repeal':
        pol.implement_reform(noctc_reform)
    # Calculate. This is needed to calculate the revenue-neutral UBI.
    calc = tc.Calculator(records=recs, policy=pol, verbose=False)
    calc.advance_to_year(year)
    calc.calc_all()
    # Create DataFrame and add identifiers.
    df = calc.dataframe(cols)
    # Add weighted sums.
    df['s006_m'] = df.s006 / 1e6
    df['expanded_income_m'] = df.s006_m * df.expanded_income
    # Add identifier.
    df['ctc_treatment'] = ctc_treatment
    # What's the column for the ID?
    df['id'] = df.index
    return df

In [9]:
scenarios = pd.concat([
    static_calc(recs, ctc_treatment='keep'),
    static_calc(recs, ctc_treatment='repeal'),
    static_calc(recs, ctc_treatment='y2017')])

## Preprocess

In [10]:
tu = scenarios.pivot_table(values=['aftertax_income', 'dep_credit', 'c07220',
                                   'c11070'],
                           index='id', columns='ctc_treatment').reset_index()
tu

id aftertax_income                         c07220         \
ctc_treatment                    keep     repeal      y2017     keep repeal   
0                   0        42278.74   42278.74   42278.74     0.00    0.0   
1                   1        20460.86   19060.86   20060.86     0.00    0.0   
2                   2        93093.50   93093.50   93093.50     0.00    0.0   
3                   3        25068.33   25068.33   25068.33     0.00    0.0   
4                   4        41630.21   41630.21   41630.21     0.00    0.0   
5                   5       119695.80  119695.80  119695.80     0.00    0.0   
6                   6        94751.99   94751.99   94751.99     0.00    0.0   
7                   7        28898.00   28898.00   28898.00     0.00    0.0   
8                   8        66179.87   66179.87   66179.87     0.00    0.0   
9                   9       101668.99   95668.99   98668.99  1397.32    0.0   
10                 10         9215.21    9215.21    9215.21     0.00    0.0   
11                 11        30790.72   30790.72   30790.72     0.00    0.0   
12                 12        30495.95   30495.95   30495.95     0.00    0.0   
13                 13        27068.44   27068.44   27068.44     0.00    0.0   
14                 14       132236.52  130236.52  130236.52  1400.00    0.0   
15                 15        98958.27   98958.27   98958.27     0.00    0.0   
16                 16        36152.01   36152.01   36152.01     0.00    0.0   
17                 17        66020.01   62020.01   64020.01  2800.00    0.0   
18                 18       134327.63  134327.63  134327.63     0.00    0.0   
19                 19        28510.88   28510.88   28510.88     0.00    0.0   
20                 20       157381.07  157381.07  157381.07     0.00    0.0   
21                 21        58950.98   58950.98   58950.98     0.00    0.0   
22                 22        40657.06   40657.06   40657.06     0.00    0.0   
23                 23        52219.71   52219.71   52219.71     0.00    0.0   
24                 24        65252.35   65252.35   65252.35     0.00    0.0   
25                 25        30767.10   30767.10   30767.10     0.00    0.0   
26                 26        18580.74   18580.74   18580.74     0.00    0.0   
27                 27        30166.59   30166.59   30166.59     0.00    0.0   
28                 28        24767.52   24767.52   24767.52     0.00    0.0   
29                 29       167357.98  167357.98  167357.98     0.00    0.0   
...               ...             ...        ...        ...      ...    ...   
456435         456435       308847.93  308847.93  308847.93     0.00    0.0   
456436         456436       259078.28  259078.28  259078.28     0.00    0.0   
456437         456437       115485.69  115485.69  115485.69     0.00    0.0   
456438         456438       450772.04  450772.04  450772.04     0.00    0.0   
456439         456439        64465.97   64465.97   64465.97     0.00    0.0   
456440         456440        82125.51   82125.51   82125.51     0.00    0.0   
456441         456441        65408.17   65408.17   65408.17     0.00    0.0   
456442         456442        32454.30   32454.30   32454.30     0.00    0.0   
456443         456443       101150.08  101150.08  101150.08     0.00    0.0   
456444         456444       749320.77  749320.77  749320.77     0.00    0.0   
456445         456445        29475.28   29475.28   29475.28     0.00    0.0   
456446         456446        20193.62   20193.62   20193.62     0.00    0.0   
456447         456447        49784.11   49784.11   49784.11     0.00    0.0   
456448         456448        22239.20   22239.20   22239.20     0.00    0.0   
456449         456449        19674.31   19674.31   19674.31     0.00    0.0   
456450         456450         2867.15    2867.15    2867.15     0.00    0.0   
456451         456451        64886.09   64886.09   64886.09     0.00    0.0   
456452         456452        19765.48   19765.48   19765.48     0.00 

In [11]:
tu.columns = ['id', 
              'afti_keep', 'afti_repeal', 'afti_y2017',
              'c07220_keep', 'c07220_repeal', 'c07220_y2017',
              'c11070_keep', 'c11070_repeal', 'c11070_y2017',
              'dc_keep', 'dc_repeal', 'dc_y2017']

In [12]:
# Dimensions based on tax unit and baseline.
base_aftiq = scenarios.loc[scenarios.ctc_treatment == 
                           'keep'].drop('ctc_treatment', axis=1)

In [13]:
tu = pd.merge(tu, base_aftiq, on='id')

In [14]:
tu['y2017_vs_repeal'] = np.where(tu.afti_y2017 > tu.afti_repeal, 'Higher',
                                 np.where(tu.afti_y2017 < tu.afti_repeal,
                                          'Lower', 'Same'))
tu['has_n24'] = tu.n24 > 0
tu['has_e18400'] = tu.e18400 > 0
tu['n24_lt_nu18'] = tu.n24 < tu.nu18
tu['records'] = 1

In [15]:
tu_pos = tu[tu.y2017_vs_repeal == 'Higher']
tu_neg = tu[tu.y2017_vs_repeal == 'Lower']

## Analysis

In [16]:
tu.pivot_table(index='y2017_vs_repeal', values=['records', 's006_m'],
               aggfunc=sum)

,records,s006_m
y2017_vs_repeal,,
Higher,86149,34.13
Same,370316,135.76


In [17]:
x = tu.pivot_table(index='y2017_vs_repeal', columns='has_n24',
                   values=['records', 's006_m', 'expanded_income_m'], 
                   aggfunc=sum)
# There's a better way to do this using slices.
x['mean_expanded_income_false'] = x.expanded_income_m[0] / x.s006_m[0]
x['mean_expanded_income_true'] = x.expanded_income_m[1] / x.s006_m[1]
x

expanded_income_m             records           s006_m         \
has_n24                     False      True     False     True   False   True   
y2017_vs_repeal                                                                 
Higher                        NaN  2.26e+06       NaN  86149.0     NaN  34.13   
Same                     9.73e+06  1.89e+06  307802.0  62514.0  124.03  11.73   

                mean_expanded_income_false mean_expanded_income_true  
has_n24                                                               
y2017_vs_repeal                                                       
Higher                                 NaN                  66253.74  
Same                              78481.42                 161433.24

In [18]:
x = tu.pivot_table(index='y2017_vs_repeal', columns='has_e18400',
                   values=['records', 's006_m', 'expanded_income_m'], 
                   aggfunc=sum)
# There's a better way to do this using slices.
x['mean_expanded_income_false'] = x.expanded_income_m[0] / x.s006_m[0]
x['mean_expanded_income_true'] = x.expanded_income_m[1] / x.s006_m[1]
x

expanded_income_m           records         s006_m          \
has_e18400                  False      True   False    True  False    True   
y2017_vs_repeal                                                              
Higher                   14869.15  2.25e+06    2823   83326   0.21   33.92   
Same                    279195.49  1.13e+07   21672  348644   7.92  127.84   

                mean_expanded_income_false mean_expanded_income_true  
has_e18400                                                            
y2017_vs_repeal                                                       
Higher                            70899.40                  66225.01  
Same                              35245.98                  88772.81

In [19]:
print(('The average people per tax unit among negative tax units is {:0.2f} '
       'vs. {:1.2f} among positive tax units.').format(
    weighted_mean(tu_neg, 'XTOT'), weighted_mean(tu_pos, 'XTOT')))

The average people per tax unit among negative tax units is nan vs. 3.22 among positive tax units.


/home/mghenis/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


### No tax units have higher aftertax_income without CTC than with 2018 CTC

In [20]:
print(('{:0.0f} tax units have higher after-tax income without CTC than '
       'with 2018 CTC').format(tu[tu.afti_repeal > tu.afti_keep].shape[0]))

0 tax units have higher after-tax income without CTC than with 2018 CTC


## Create tax records

Start with a record with unexpected result, then re-test after simplifying the record (zeroing out various inputs) until a minimal case is identified.

Without a way to convert Records to dataframes and back, or to index Records, unclear how to do this.

In [21]:
import os
data = os.path.join(tc.Records.CUR_PATH, 'cps.csv.gz')

In [22]:
test = pd.read_csv(data).iloc[[351322, 137218, 139057, 202577]]

Examine four test records: first two with `n24 == 0` and second two with `n24 > 0`.

*Split into two cells to see all columns.*

In [23]:
test.transpose().iloc[:30]

,351322,137218,139057,202577
age_head,23,49,33,45
age_spouse,0,52,31,0
e00200p,106560,54390,288404,151125
e00900p,0,0,0,0
e02100p,0,0,0,0
e00200s,0,86117,28288,0
e00900s,0,0,0,0
e02100s,0,0,0,0
a_lineno,1,1,1,1
e00600,0,6540,0,0


In [24]:
test.transpose().iloc[30:]

,351322,137218,139057,202577
housing_ben,0,0,0,0
wic_ben,0,0,0,0
XTOT,2,3,4,3
filer,1,1,1,1
FLPDYR,2012,2014,2014,2013
MARS,1,2,2,1
e01100,213,1090,0,0
e01400,0,0,0,0
e03300,0,0,0,0
e03270,0,0,0,0


### Alter records

Zero out interest.

In [25]:
test_alt = test.copy()

In [26]:
INTEREST_COLS = ['e01100', 'e01400', 'e03300', 'e03270', 'e20400', 'e32800',
                 'e19200', 'e18500', 'e03240', 'e17500', 'e18400', 'e00900',
                 'e00650', 'e00300', 'e00400', 'e01700', 'e19800', 'e20100',
                 'e03210', 'e03150', 'e02300']

In [27]:
test_alt[INTEREST_COLS] = 0

Examine records on attributes that aren't completely zero.

In [28]:
test_alt.loc[:, (test_alt != 0).any(axis=0)].transpose()

,351322,137218,139057,202577
age_head,23,49,33,45
age_spouse,0,52,31,0
e00200p,106560,54390,288404,151125
e00200s,0,86117,28288,0
a_lineno,1,1,1,1
e00600,0,6540,0,0
s006,548,46,11,37
h_seq,35570,78900,80590,20484
ffpos,1,1,1,1
fips,29,8,35,39


### Calculate altered records

In [29]:
recs_alt = tc.Records.cps_constructor(data=test_alt)

In [30]:
scenarios_alt = pd.concat([
    static_calc(recs_alt, ctc_treatment='keep'),
    static_calc(recs_alt, ctc_treatment='y2017'),
    static_calc(recs_alt, ctc_treatment='repeal')])

### Preprocess

In [31]:
tu_alt = scenarios_alt.pivot_table(values='aftertax_income',
                                   index='id', 
                                   columns='ctc_treatment').reset_index()
tu_alt

ctc_treatment,id,keep,repeal,y2017
0,0,93488.50,92988.50,92988.50
1,1,134609.47,134109.47,134109.47
2,2,279754.07,277254.07,277254.07
3,3,133009.14,130509.14,130509.14


In [32]:
tu_alt.columns = ['id', 'afti_keep', 'afti_repeal', 'afti_y2017']

In [33]:
# Dimensions based on tax unit and baseline.
base_aftiq_alt = scenarios_alt.loc[scenarios_alt.ctc_treatment == 
                                   'keep'].drop('ctc_treatment', axis=1)

In [34]:
tu_alt = pd.merge(tu_alt, base_aftiq_alt, on='id')

In [35]:
tu_alt['y2017_vs_repeal'] = (
    np.where(tu_alt.afti_y2017 > tu_alt.afti_repeal, 'Higher',
             np.where(tu_alt.afti_y2017 < tu_alt.afti_repeal,
                      'Lower', 'Same')))
tu_alt['has_n24'] = tu_alt.n24 > 0
tu_alt['has_e18400'] = tu_alt.e18400 > 0
tu_alt['n24_lt_nu18'] = tu_alt.n24 < tu_alt.nu18
tu_alt['records'] = 1

In [36]:
tu_pos_alt = tu_alt[tu_alt.y2017_vs_repeal == 'Higher']
tu_neg_alt = tu_alt[tu_alt.y2017_vs_repeal == 'Lower']

### Analysis

In [37]:
tu_alt.pivot_table(index='y2017_vs_repeal', values=['records', 's006_m'],
                   aggfunc=sum)

,records,s006_m
y2017_vs_repeal,,
Same,4,169.89


In [38]:
tu_alt

,id,afti_keep,afti_repeal,afti_y2017,s006,aftertax_income,expanded_income,n24,nu18,e18400,...,c07220,c11070,dep_credit,s006_m,expanded_income_m,y2017_vs_repeal,has_n24,has_e18400,n24_lt_nu18,records
0,0,93488.50,92988.50,92988.50,1.45e+08,93488.50,132925.31,0.0,0.0,0.0,...,0.0,0.0,500.0,144.70,1.92e+07,Same,False,False,False,1
1,1,134609.47,134109.47,134109.47,1.23e+07,134609.47,183178.39,0.0,1.0,0.0,...,0.0,0.0,500.0,12.29,2.25e+06,Same,False,False,True,1
2,2,279754.07,277254.07,277254.07,2.94e+06,279754.07,382512.22,1.0,2.0,0.0,...,1400.0,0.0,1100.0,2.94,1.13e+06,Same,True,False,True,1
3,3,133009.14,130509.14,130509.14,9.97e+06,133009.14,185842.95,1.0,1.0,0.0,...,1400.0,0.0,1100.0,9.97,1.85e+06,Same,True,False,False,1


In [39]:
tu_alt.transpose()

,0,1,2,3
id,0,1,2,3
afti_keep,9.3e+04,1.3e+05,2.8e+05,1.3e+05
afti_repeal,9.3e+04,1.3e+05,2.8e+05,1.3e+05
afti_y2017,9.3e+04,1.3e+05,2.8e+05,1.3e+05
s006,1.4e+08,1.2e+07,2.9e+06,1e+07
aftertax_income,9.3e+04,1.3e+05,2.8e+05,1.3e+05
expanded_income,1.3e+05,1.8e+05,3.8e+05,1.9e+05
n24,0,0,1,1
nu18,0,1,2,1
e18400,0,0,0,0
